In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d aunanya875/suicidal-tweet-detection-dataset
!chmod 600 /root/.kaggle/kaggle.json

###El proposito de este programa es usar el modelo de ML de naive bayes para poder crear un modelo de prediccion de la potencialidad de comerter suicidio basado en los tweets. El modelo usa la base de datos que a continuación se describe.

###Este conjunto de datos proporciona una colección de tweets junto con una anotación que indica si cada tweet está relacionado con el suicidio o no. El objetivo principal de este conjunto de datos es facilitar el desarrollo y la evaluación de modelos de aprendizaje automático para la clasificación de tweets que expresan sentimientos suicidas o no.

###Columnas:

###Tweet: Esta columna contiene el contenido textual de los tweets obtenidos de diversas fuentes. Los tweets cubren una amplia gama de temas, emociones y expresiones.
###Suicidio: Esta columna proporciona anotaciones que indican la clasificación de los tweets. Los valores posibles son:
###Publicación no suicida: esta etiqueta se asigna a tweets que no expresan ningún sentimiento o intención suicida.
###Publicación potencial de suicidio: esta etiqueta se asigna a tweets que muestran indicios de pensamientos, sentimientos o intenciones suicidas.



Importamos las librerías que usaremos y que son el pan de cada día en la ciencia de datos, excepto zipfile, esa es para los archivos Zip.

In [ ]:
import pandas as pd
import numpy as np
import zipfile

In [ ]:
with zipfile.ZipFile("/content/suicidal-tweet-detection-dataset.zip", "r" ) as zip_ref:
  zip_ref.extractall("/content/")

In [ ]:
df = pd.read_csv("/content/Suicide_Ideation_Dataset(Twitter-based).csv")

In [ ]:
df.shape

(1787, 2)

In [ ]:
df.head()

,Tweet,Suicide
0,making some lunch,Not Suicide post
1,@Alexia You want his money.,Not Suicide post
2,@dizzyhrvy that crap took me forever to put to...,Potential Suicide post
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,Not Suicide post
4,Trying out &quot;Delicious Library 2&quot; wit...,Not Suicide post


In [ ]:
data_2=df.copy()

Podemos ver que hay dos filas con datos null, estas las borraremos pues no aportan nada y al tratarse de archivos de tipo cadena no son tan fáciles de remplazar

In [ ]:
data_2.isnull().sum()

Tweet      2
Suicide    0
dtype: int64

In [ ]:
data_2 = data_2.dropna()
data_2.loc[:, 'Tweet'] = data_2['Tweet'].astype(str)

In [ ]:
data_2.isnull().sum()

Tweet      0
Suicide    0
dtype: int64

Cambiaremos en la columna de suicidio las varibles de "Not suicide post" por 0 y "Potencial Suicide post" por 1.

In [ ]:
data_2["Suicide"].replace("Not Suicide post", 0, inplace = True)
data_2["Suicide"].replace("Potential Suicide post ", 1, inplace = True)

Verificamos que en la columna de Suicidio solo hay dos valores: 1 y 0.

In [ ]:
data_2["Suicide"].unique()

array([0, 1])

Como usted ya lo pudo anticipar, el análisis tiene que ser por texto, entonces procederemos a limpiar los elementos del tweet que no sirven al análisis como los emojis y símbolos que no están en el alfabeto.

In [ ]:
data_2

,Tweet,Suicide
0,making some lunch,0
1,@Alexia You want his money.,0
2,@dizzyhrvy that crap took me forever to put to...,1
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,0
4,Trying out &quot;Delicious Library 2&quot; wit...,0
...,...,...
1782,i have forgotten how much i love my Nokia N95-1,0
1783,Starting my day out with a positive attitude! ...,0
1784,"@belledame222 Hey, it's 5 am...give a girl som...",0
1785,2 drunken besties stumble into my room and we ...,0


In [ ]:
!pip install demoji  #Esta librería nos ayudará en la tarea de eliminar los emojis

In [ ]:
import demoji
demoji.download_codes()

In [ ]:
def eliminar_emo(texto):
  return demoji.replace(texto,'')

Aquí podemos ver como se esta generando la limpieza de los tweets se están dejando las palabras que le dan sentido a la oración.

In [ ]:
data_2["Tweet"]=data_2["Tweet"].apply(eliminar_emo)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'[³áµ¹¸·â’Êæ±ËïÃ¯Â½ð­­­ª¾¶º¤¼´ó§¢¥§¼]', '', regex=True)
data_2["Tweet"] = data_2["Tweet"].str.replace(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', regex=True)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'\[|\]', '', regex=True)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'[¬*"°¿^|£~–=¦¡/()%$.,!?@&#:;_\\\-+]', '', regex=True)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'\"|\`', '', regex=True)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'\"|\'', '', regex=True)

In [ ]:
data_2.head() #Damos una vista previa de los datos ya limpios

,Tweet,Suicide
0,making some lunch,0
1,Alexia You want his money,0
2,dizzyhrvy that crap took me forever to put tog...,1
3,jnaylor kiwitweets Hey Jer Since when did you ...,0
4,Trying out quotDelicious Library 2quot with mi...,0


Utilizamos la librería "re" para crear una función que nos permita visualizar los caracteres que no están en el alfabeto, tanto en minúsculas como en mayúsculas. Esto nos servirá para obtener una representación de los emojis y caracteres que estaban presentes en el texto.

In [ ]:
import re
def buscar_no_letras(texto):
    no_letras_pattern = re.compile(r'[^a-zA-Z]')
    caracteres_no_letras = no_letras_pattern.findall(texto)
    return caracteres_no_letras

In [ ]:
pd.DataFrame(data_2["Tweet"].apply(buscar_no_letras))

,Tweet
0,"[ , ]"
1,"[ , , , ]"
2,"[ , , , , , , , , , , , , , , ]"
3,"[ , , , , , , , , ]"
4,"[ , , , , 2, , , , , , , , , , , ..."
...,...
1782,"[ , , , , , , , , , 9, 5, 1]"
1783,"[ , , , , , , , , , , , ]"
1784,"[2, 2, 2, , , , 5, , , , , , , ]"
1785,"[2, , , , , , , , , , , , , , , ..."


Aquí es donde el proceso se vuelve interesante. Ahora necesitamos vectorizar nuestros tweets, y para lograrlo utilizaremos "TfidfVectorizer". Esta herramienta asignará pesos a nuestras palabras y ayudará a capturar mejor el contexto de las oraciones. El resultado será un DataFrame en el que las palabras se convierten en columnas, las filas representan los tweets y sus ponderaciones correspondientes.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
Danni_Flow = TfidfVectorizer() #La variable la iba a llamar vector, pero todos sabemos que danni flow se parece a vector
V_TWEET = Danni_Flow.fit_transform(data_2['Tweet'])

In [ ]:
P_F= pd.DataFrame(V_TWEET.toarray(), columns=Danni_Flow.get_feature_names_out())

Aquí es donde empezamos a importar el algoritmo de naive bayes y separamos el dataset para entrenarlo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
X_train, X_test, Y_train, Y_test=train_test_split(
    P_F, data_2["Suicide"], test_size=0.3, random_state=42
)

Utilizamos la distribución multinomial porque es la más adecuada para los casos en los que se requiere categorización.

In [ ]:
nm_class=MultinomialNB()
nm_class.fit(X_train,Y_train)

In [ ]:
Y_Pred=nm_class.predict(X_test)

Podemos observar que el modelo presenta un excelente desempeño con una exactitud del 0.91, lo cual sin duda indica que es una aproximación sólida y cuenta con un fundamento sólido.

In [ ]:
accuracy = accuracy_score(Y_test, Y_Pred)
report = classification_report(Y_test, Y_Pred)
print(f"Exactitud: {accuracy:.2f}")
print("Reporte de clasificación:\n", report)

Exactitud: 0.91
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93       338
           1       0.96      0.80      0.87       198

    accuracy                           0.91       536
   macro avg       0.92      0.89      0.90       536
weighted avg       0.92      0.91      0.91       536



Procedemos a generar la validación cruzada para poder ver como se comporta el modelo en distintas particiones del dataset.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(MultinomialNB(), P_F, data_2["Suicide"], scoring="accuracy", cv=10)

mean_score = scores.mean()
std_score = scores.std()
print(scores)
print("Media de puntajes:", mean_score)
print("Desviación estándar de puntajes:", std_score)

[0.88268156 0.91620112 0.93854749 0.87709497 0.93296089 0.91011236
 0.93258427 0.92134831 0.91011236 0.91011236]
Media de puntajes: 0.9131755696440902
Desviación estándar de puntajes: 0.019358931235342106


In [ ]:
prediccion_X=pd.DataFrame({
    "Y_pred" : nm_class.predict(P_F),
    "Y_original" : data_2["Suicide"],
    "Probabilidad": nm_class.predict_proba(P_F).T [1],
    "Complemento": nm_class.predict_proba(P_F).T [0]
})

Aquí tenemos un dataframe donde  podemos ver las probabilidades de que sea catalogado como un tweet de potencial de suicidio y su complemento.

In [ ]:
prediccion_X

,Y_pred,Y_original,Probabilidad,Complemento
0,0,0,0.143420,0.856580
1,0,0,0.211432,0.788568
2,1,1,0.720608,0.279392
3,0,0,0.177908,0.822092
4,0,0,0.141080,0.858920
...,...,...,...,...
1782,0,0,0.289373,0.710627
1783,0,0,0.128508,0.871492
1784,0,0,0.107359,0.892641
1785,0,0,0.312196,0.687804


Debemos tener en cuenta que nuestro modelo tiene margen de mejora en la reducción de falsos negativos, ya que la precisión en los falsos positivos fue superior. Esta situación puede resolverse eficazmente al incrementar la cantidad de datos en los cuales se pueda identificar contenido que promueva el suicidio. Con un conjunto de datos más amplio y representativo, el algoritmo podrá entrenarse de manera más efectiva, lo que resultará en una mayor capacidad para detectar casos de incitación al suicidio.

In [ ]:
Falsos_neg=prediccion_X[(prediccion_X['Y_original']==1) & (prediccion_X['Y_pred']==0)]
Falsos_pos=prediccion_X[(prediccion_X['Y_original']==0) & (prediccion_X['Y_pred']==1)]

In [ ]:
Falsos_pos["Y_pred"].sum(), Falsos_neg["Y_original"].sum()

(8, 54)

Podemos concluir de manera satisfactoria que el modelo se ha adaptado a los datos disponibles y ha demostrado un desempeño prometedor. Sin embargo, es importante reconocer que siempre existe margen para mejoras. Esto refleja una ley natural de la vida: todo es perfectible y continuamente se puede evolucionar.

Aunque el modelo ha logrado una gran aproximación en la detección de casos relevantes, es evidente que su rendimiento podría beneficiarse al expandir el número de datos disponibles. Al aumentar el tamaño y la diversidad del conjunto de datos, el modelo tendrá la oportunidad de aprender de una gama más amplia de situaciones y matices. Este enfoque tiene el potencial de elevar aún más su eficacia y precisión en la detección de contenido que involucre incitación al suicidio.